In [3]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *
from spytial.annotations import flag

There are many ways to represent graphs, including explicitly encoding the edges in the graph node, adjacency matrices, adjacency lists, edge lists.

In CLRS (Ch. 22) the default is the adjacency-list representation.

In [15]:
@attribute(field='key')
class GNode:
    def __init__(self, key):
        self.key = key


#@flag(name="hideDisconnected")
@inferredEdge(selector='{u : GNode, w : int, v : GNode | (some t : tuple | t.t0 = u and t.t1 = v and t.t2 = w) }', name='edge')
@hideAtom(selector='list + tuple + Graph + int')
class Graph:
    def __init__(self):
        self.adj = []  # adjacency list representation

    def addEdge(self, u: GNode, v: GNode, w: int):
        self.adj.append((u, v, w))


# Now lets define some nodes
x = GNode("x")
y = GNode("y")
z = GNode("z")

g = Graph()
g.addEdge(x, y, 5)
g.addEdge(y, z, 10)
g.addEdge(z, z, 15)
g.addEdge(z, x, 20)

diagram(g)




# Also UNWEIGHTED Graph

![unweighted-dir-graph](/Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/clrs/img/directed-graph.png)


In [ ]:
## Here, we show how an unweighted directed graph could be built from an adjacency matrix.

adj_matrix = [[0, 1, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1, 1],
              [0, 1, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1]]


# These are 1 indexed instead of 0 indexed.



## Relational query to python matrices into rels between indices
ad = inferredEdge(selector="{i, j : int | (some xs : list | (some ys : list |  (xs->i->ys) in idx and (ys -> j -> ({x : int | @num:x = 1})  in idx  )))}", name="e")(adj_matrix)
ad = hideAtom(selector="list-int")(ad)
diagram(ad)

# Some kind of bug here where int is ~in~ list?